# Subject Subset

Curate datasets

----
```
author:     Zach Wolpe
email:      zachcolinwolpe@gmail.com
date:       21 January 2022
```
----

In [87]:
import sys
sys.path.append('../process data')
from dependencies import *

In [88]:
# load -------------------------------------*
loc = '/Users/zachwolpe/Documents/Production/Dynocog/Python Implementation/final instance/model-free analysis/final_dataframes'
wcst_data   = pd.read_pickle(loc + '/wcst_raw_data.pkl')
wcst        = pd.read_pickle(loc + '/wcst.pkl')
covariates  = pd.read_pickle(loc + '/covariates.pkl')
wcst

,participant,reward,status,action,rule,matching_rule,n_t
0,816404.0,1,1,2,shape,number,0
1,816404.0,1,1,2,shape,number,1
2,816404.0,1,1,2,shape,number,2
3,816404.0,1,1,2,shape,number,3
4,816404.0,1,1,2,shape,number,4
...,...,...,...,...,...,...,...
27395,684712.0,1,1,2,color,number,95
27396,684712.0,1,1,2,color,number,96
27397,684712.0,1,1,2,color,number,97
27398,684712.0,1,1,2,color,number,98


# Subset data

1. Top 100
2. Random set

In [93]:
# top 100 participants ------------------------------------------------------------------------------------------------------------*
top_subjects = covariates[['wcst_accuracy']].reset_index().sort_values('wcst_accuracy', ascending=False).participant.values[:100]
wcst_top_100 = wcst.loc[wcst.participant.isin(top_subjects)]


# random 100 lam > 40 --------------------------------------------------------------------------------------------------------------*
np.random.seed(0)
lam             = 0.40
subjects        = covariates.index[covariates.wcst_accuracy > lam]
subjects        = np.random.choice(np.unique(subjects), 100, replace=False)
wcst_random_set = wcst.loc[wcst.participant.isin(subjects)]

# save dataframes ------------------------------------------------------------------------------------------------------------------*
sav_loc = '/Users/zachwolpe/Documents/Production/Dynocog/Python Implementation/final instance/data objects - processed/'
wcst_top_100.to_pickle(sav_loc      + 'df_wcst_top_100.pkl')
wcst_top_100.to_csv(sav_loc         + 'df_wcst_top_100.csv')
wcst_random_set.to_pickle(sav_loc   + 'wcst_random_set.pkl')
wcst_random_set.to_csv(sav_loc      + 'wcst_random_set.csv')


# pyStan data object


In [ ]:
# top 100
def pyStan_data_object(wcst_set, sav_loc, save=True):
    wcst_set.action   = wcst_set.action.astype(int)
    wcst_set.reward   = wcst_set.reward.astype(int)
    action_matrix = wcst_set[['participant' ,'n_t', 'action']].pivot(index='participant', columns='n_t')
    reward_matrix = wcst_set[['participant' ,'n_t', 'reward']].pivot(index='participant', columns='n_t')
    data_object = {
        'n_s':    reward_matrix.shape[0],
        'n_t':    reward_matrix.shape[1],
        'action': action_matrix,
        'reward': reward_matrix+1
    }
    # import pickle
    if save:
        with open(sav_loc, "wb") as f:
            pickle.dump(data_object, f, protocol=-1)
    return data_object




pyStan_data_object(wcst_top_100,    sav_loc+'pyStan_data_object_wcst_top_100.pkl')
pyStan_data_object(wcst_random_set, sav_loc+'pyStan_data_object_wcst_random_set.pkl')

# Psychological & Demographic Covariates

In [95]:
psychological_covariates    = covariates[['wcst_RT', 'fitts_mean_deviation', 'nback_status', 'nback_reaction_time_ms', 'navon_perc_correct', 'navon_reaction_time_ms', 'corsi_block_span']]
demographic_covariates      = covariates[['demographics_mean_reation_time_ms', 'demographics_income_a', 'demographics_computer_hours_a', 'demographics_age_a']]

# save dataframes ------------------------------------------------------------------------------------------------------------------*
sav_loc = '/Users/zachwolpe/Documents/Production/Dynocog/Python Implementation/final instance/data objects - processed/'
psychological_covariates.to_csv(sav_loc + 'psychological_covariates.csv')
demographic_covariates.to_csv(sav_loc   + 'demographic_covariates.csv')

In [71]:
def compute_X_matrix(df1, df2, params):
  theta = []
  theta.append('participant')
  [theta.append(i) for i in params]
  return df1.set_index('participant').join(df2[theta])[params].drop_duplicates()


In [204]:

def covariate_set(df1, df2):
    df2 = df2.reset_index()
    return df1.merge(df2, how='left', on='participant')[df2.columns].drop_duplicates()

psy_covars_top100 = covariate_set(wcst_top_100,     psychological_covariates)
dem_covars_top100 = covariate_set(wcst_top_100,     demographic_covariates)
psy_covars_ran100 = covariate_set(wcst_random_set,  psychological_covariates)
dem_covars_ran100 = covariate_set(wcst_random_set,  demographic_covariates)


psy_covars_top100.to_csv(sav_loc + 'psy_covars_top100.csv')
dem_covars_top100.to_csv(sav_loc + 'dem_covars_top100.csv')
psy_covars_ran100.to_csv(sav_loc + 'psy_covars_ran100.csv')
dem_covars_ran100.to_csv(sav_loc + 'dem_covars_ran100.csv')
